In [45]:
import pandas as pd 
import requests
import json

In [ ]:

countries = {
    "Rwanda": "RW",
    "Nigeria": "NG",
    "South Africa": "ZA",
    "Kenya": "KE",
}

tmdb_url = "https://api.themoviedb.org/3/discover/movie"
tmdb_api_key = "49c51d9ce999caed0e75e5ae67748947"
omdb_url = "https://www.omdbapi.com/?i=tt3896198&apikey=42316941"  

All_movies = [] 

for country_name, country_code in countries.items():
    #print(f"Getting movies from {country_name}...")

    tmdb_params = {
        "api_key": tmdb_api_key,
        "with_origin_country": country_code,
        "sort_by": "popularity.desc",
        "with_original_language": "en",
        #"page": 1
    }

    response = requests.get(tmdb_url, params=tmdb_params)
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        data = response.json()
        movies = data.get("results", [])

        for movie in movies:
            title = movie.get("title")
            if title:
               # print("  -", title)

             
                omdb_response = requests.get(omdb_url).json()

                movie_info = {
                    'Title': title,
                    'Country': country_name,
                    'Director': omdb_response.get('Director'),
                    'Year': omdb_response.get('Year'),
                    'IMDb Rating': omdb_response.get('imdbRating')
                }

                All_movies.append(movie_info)  
    else:
        print("  Failed to fetch data.")

    print()  

df = pd.DataFrame(All_movies)
print(df)


df.to_csv("african_movies_combined.csv", index=False)


In [ ]:
def african_movies_data(tmdb_api_key, omdb_api_key):
    countries = {
        "Rwanda": "RW",
        "Nigeria": "NG",
        "South Africa": "ZA",
        "Kenya": "KE",
    }
    All_Movies = []

    for country_name, country_code in countries.items():
        print(f"Getting movies from {country_name}...")

        tmdb_url = "https://api.themoviedb.org/3/discover/movie"
        tmdb_params = {
            "api_key": tmdb_api_key,
            "with_origin_country": country_code,
            #"sort_by": "popularity.desc",
            #"with_original_language": "en",
            #"page": 1
        }

        response = requests.get(tmdb_url, params=tmdb_params)
        if response.status_code != 200:
            print(f"Failed to get data for {country_name}")
            continue

        data = response.json()
        movies = data.get("results", [])

        for movie in movies:
            title = movie.get('title')
            if not title:
                continue

            omdb_url = "https://www.omdbapi.com/"
            omdb_params = {
                'apikey': omdb_api_key,
                't': title
            }

            omdb_response = requests.get(omdb_url, params=omdb_params).json()
            
            movie_info = {
                'Title': title,
                'Country': country_name,
                'Director': omdb_response.get('Director'),
                'Released': omdb_response.get('Released'),
                'IMDb Rating': omdb_response.get('imdbRating')
            }

            All_Movies.append(movie_info)

    df = pd.DataFrame(All_Movies)

    df['IMDb Rating'] = df['IMDb Rating'].replace(['N/A', '', None], 'Not Rated').fillna('Not Rated')
    df['Released'] = df['Released'].replace("Not Available", None)
    df['Released'] = pd.to_datetime(df['Released'], errors='coerce')
    df['Director'] = df['Director'].replace(['N/A', None], 'Unknown').fillna('Unknown')
    df.to_csv("african_movies.csv", index=False)
    print("Saved all movies data to 'african_movies.csv'.")
    print(df)


tmdb_api_key = "49c51d9ce999caed0e75e5ae67748947"
omdb_api_key = "42316941"
african_movies_data(tmdb_api_key, omdb_api_key)


Getting movies from Rwanda...
Getting movies from Nigeria...


In [69]:
print(df.isnull().sum()) 

Title          0
Country        0
Director       0
Year           0
IMDb Rating    0
dtype: int64
